# Mobility and amenities calculation

*Authour: Liubov, Collaborators: Bastian, Marc, Vero* 

**Algorithm**
1. Load the data on mobility from Open Humans, data formats are described https://www.openhumans.org/data-management/datatypes/4 
2. Given the location we extract information about name of the space and give suggestions for places (amenities) around it. We use osmnx package for that.

# 1. Data loading
First we load mobility data, make analysis and load the dataframe of it. 

In [8]:
# function to read the data 

import os
import json
import requests
from datetime import datetime
from collections import defaultdict
import pandas as pd
import numpy as np
import seaborn as sns


def dataframe_from_json(moves_data):
    for datapoint in moves_data:
        # we need to have observed segments for that day. If moves wasn't running we ignore the day
        if datapoint['segments'] != None:
            # did we stay in a place that day and did we walk that day?
            has_places = False
            walked = False
            for i in datapoint['segments']:
                if i['type'] == 'place':
                        # yes, we were in one place w/o moving around too much, we can keep this day
                        has_places = True
                        
            # is this day in our date range of interest and has data?
            if datapoint['summary'] != None and has_places and datetime.strptime(datapoint['date'],"%Y%m%d") > datetime.strptime(DATARANGE_START,"%Y-%m-%d"):
                moves_processed_data['date'].append(datapoint['date'])
                for activity in datapoint['summary']:
                    if activity['activity'] == 'walking':
                        moves_processed_data['steps'].append(activity['steps'])
                        moves_processed_data['distance'].append(activity['distance'])
                        walked = True
                        
                # in case of not walking, step count is zero
                if not walked:
                    moves_processed_data['steps'].append(0)
                    moves_processed_data['distance'].append(0)  
                    
                # distribution of stops lengths
                stops_distrib = duration_stop_distribution(datapoint['segments'])
                moves_processed_data['duration'].append(stops_distrib)
                location = longest_daily_location(datapoint['segments'])
                moves_processed_data['lat'].append(location['lat'])
                moves_processed_data['lon'].append(location['lon'])    
    
    
    
    #Now that we have all of the data we can convert it into a single pandas dataframe for easier processing and visualization
    
    moves_dataframe = pd.DataFrame(data={
        'date': moves_processed_data['date'],
        'steps': moves_processed_data['steps'],
        'distance': moves_processed_data['distance'],
        'latitude': moves_processed_data['lat'],
        'longitude': moves_processed_data['lon']
    })   
    
    return moves_dataframe

In [11]:
# main function analyzing moves_data    
DATARANGE_START = "2016-06-01"
DATARANGE_END = "2018-05-08"

import os
import json
import requests
from datetime import datetime
from collections import defaultdict
import pandas as pd
import numpy as np
import seaborn as sns




#### sets the axis label sizes for seaborn #######################
rc={'font.size': 14, 'axes.labelsize': 14, 'legend.fontsize': 14.0, 
    'axes.titlesize': 14, 'xtick.labelsize': 14, 'ytick.labelsize': 14}
sns.set(rc=rc)
##################################################################


with open('C:/Users/lyubo/Documents/DATA_networks/mobilitydata/openhumans/moves-storyline-data98972.json') as f:
    moves_data = json.load(f)   





In [ ]:
# function to get data from json data
df1 = dataframe_from_json(moves_data)
#df2 = dataframe_from_json(moves_data2)
#df3 = dataframe_from_json(moves_data3)


print('Now data looks like...')
df1.head(10)

# 2. Analysis of amenities around the locations  
Here we analyze and read amenities around the location, which were visited frequently, or where user stayed for long time.  

In [3]:
from geopy.geocoders import Nominatim

#example location
geolocator = Nominatim(user_agent="specify_your_app_name_here")


########## Choose location which is the third visited place ##################
location = geolocator.reverse("52.509669, 13.376294") #example of third place#
print(location.address)
##############################################################################

#first location from data 
lat = df1.latitude.iloc[1]
lon = df1.longitude.iloc[1]
print(lat, lon)
#lat = 52.52437 #Berlin example
#lon = 13.41053
location = geolocator.reverse(lat, lon)
print(location.address)

latrnd = np.around(lat, decimals=4)
latfloat = np.float(lat)
print(np.around(lat, decimals=4))
print(type(latrnd))
#Geopy can calculate geodesic distance between two points using the geodesic distance or the great-circle distance, with a default of the geodesic distance available as the function geopy.distance.distance.
#Here’s an example usage of the geodesic distance:
print(type(latfloat))

Backwerk, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland
33.99882 -118.43657
El Biod, daïra Mecheria, النعامة, الجزائر
33.9988
<class 'numpy.float64'>
<class 'float'>


### Statistics for amenities
Here we calculate number of schools and other amenities in surrounding of the location.

First we plot the map of the city and plot all streets around the given location.

*Problem:*
Here we use Openstreetmap information about the maps, although this information is not complete. 

In [ ]:
import networkx as nx
import osmnx as ox
import requests
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
ox.config(use_cache=True, log_console=True)
ox.__version__
# get a graph for some city

lat1 = np.float(lat) +0.02
lat2 = np.float(lat) -0.02
lon1 = np.float(lon) +0.03
lon2 = np.float(lon) -0.03

Coor_box = [lat1, lat2, lon1, lon2]
G2 = ox.graph_from_bbox(
    *SMALL_MARINA,
    simplify=False,
    retain_all=True,
    network_type='drive',

)
fig2, ax2 = ox.plot_graph(G2, fig_height=20, fig_width=20)

# get the nearest network node to each point

good_orig_node = ox.get_nearest_node(G2, (25.070661, 55.137911), method='euclidean')
bad_orig_node = ox.get_nearest_node(G2, (25.071764, 55.138978), method='euclidean')
dest_node = ox.get_nearest_node(G2, (25.079223, 55.146241), method='euclidean')


# find the route between location of a person and a school
route = nx.shortest_path(G2, bad_orig_node, dest_node, weight='length')
fig, ax = ox.plot_graph_route(G2, route, fig_height=20, fig_width=20)

In [ ]:
import osmnx as ox
from IPython.display import Image
%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__


# configure the inline image display
img_folder = 'images'
extension = 'png'
size = 240

point = (np.float(lat), np.float(lon))
dist = 612

gdf = ox.footprints.footprints_from_point(point=point, distance=dist)
print(type(gdf))
gdf_proj = ox.project_gdf(gdf)
bbox = ox.bbox_from_point(point=point, distance=dist, project_utm=True)
fig, ax = ox.footprints.plot_footprints(gdf_proj, bgcolor='#333333', color='w', figsize=(4,4), bbox=bbox,
                            save=True, show=False, close=True, filename='paris_bldgs', dpi=90)
Image('{}/{}.{}'.format(img_folder, 'paris_bldgs', extension), height=size, width=size)

# Search for schools

We search for schools in the surroundings of the trajectory.


In [ ]:
## Search for places to contribute around trajectory 

from shapely.geometry import Point, Polygon



#lat = 52.52437 #Berlin example
#lon = 13.41053

# Create Point objects from trajectory:
lat1 = np.float(lat) +0.02
lat2 = np.float(lat) -0.02
lon1 = np.float(lon) +0.03
lon2 = np.float(lon) -0.03

p1 = Point(lat1, lon1)
p2 = Point(lat2, lon2)

# Create a Polygon

coords = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)] #working example
#coords = [(lat1, lon1), (lat1, lon2), (lat2, lon1), (lat2, lon2)]
poly = Polygon(coords)

#schools = ox.pois_from_place(place_name, amenities=['school']) #placename should be name of place str object

schools = ox.pois.osm_poi_download(polygon=poly, amenities=['school'])# north=None, south=None, east=None, west=None, timeout=180, max_query_area_size=2500000000)

# How many schools do we have in this district?
print('number of schools',len(schools))
#print(type(schools))

# Available columns and names of amenity we are searching for
#schools.columns


# Opensteetreet amenities 

(Work in progress on extracting the information around locations)


In [6]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:25];
(
node(around:60.0,13.74567157,100.53371655);
);
out body;
    """
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()
print(data)

{'version': 0.6, 'generator': 'Overpass API 0.7.55.1009 5e627b63', 'osm3s': {'timestamp_osm_base': '2019-11-27T18:10:02Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'node', 'id': 280205414, 'lat': 13.7458894, 'lon': 100.5335488}, {'type': 'node', 'id': 297713248, 'lat': 13.7457155, 'lon': 100.533643}, {'type': 'node', 'id': 373097475, 'lat': 13.7457442, 'lon': 100.5332224, 'tags': {'bench': 'yes', 'colour': 'blue', 'highway': 'bus_stop', 'name': 'สยามสแควร์', 'name:en': 'Siam Square', 'name:th': 'สยามสแควร์', 'ref': '24', 'route_ref': '15; 16; 21; 25; 40; 48; 54; 73; 73ก; 141; 204; 501; 508', 'shelter': 'no'}}, {'type': 'node', 'id': 480774962, 'lat': 13.7457073, 'lon': 100.5332908}, {'type': 'node', 'id': 480774964, 'lat': 13.7458801, 'lon': 100.5333143}, {'type': 'node', 'id': 480774965, 'lat': 13.7456915, 'lon': 100.5332792}, {'type': 'node', 'id': 1692209237, 'lat': 13.7456236, 'lo